# Kmeans1

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class KMeans:
    def __init__(self, k=2, epochs=100, ep=1e-4, seed=42):
        self.k = k
        self.ep = ep
        self.epochs = epochs
        np.random.seed(seed)
        self.centroids = None
        self.labels = None
        self.j_hist = []

    def init_centroids(self, X):
        idxs = np.random.choice(X.shape[0], self.k, replace=False)
        return X[idxs]
    
    def assign_cluster(self, X, centroids):
        assign = []
        for x in X:
            distances = [self._dist(x, pt) for pt in centroids]
            assign.append(np.argmin(distances))
        return np.array(assign)
    
    def _dist(self, a, b):
        return np.sqrt(np.sum((a - b) ** 2))
    
    def update_centroids(self, X, labels):
        centroids = np.zeros((self.k, X.shape[1]))
        for i in range(self.k):
            points = X[labels == i]
            if len(points) > 0:
                centroids[i] = points.mean(axis=0)
            else:
                centroids[i] = X[np.random.choice(X.shape[0])]
        return centroids
    
    def compute_objective(self, X, labels, centroids):
        total = 0
        for i in range(self.k):
            cluster_points = X[labels == i]
            total += np.sum((cluster_points - centroids[i]) ** 2)

        return total / X.shape[0]
    
    def fit(self, X):
        self.centroids = self.init_centroids(X)
        for i in range(self.epochs):
            labels = self.assign_cluster(X, self.centroids)
            new_centroids = self.update_centroids(X, labels)
            j = self.compute_objective(X, labels, new_centroids)
            self.j_hist.append(round(j, 4))
            if np.linalg.norm(new_centroids - self.centroids) < self.ep:
                break
            self.centroids = new_centroids
        self.labels = labels
        return self
    
    def plot_convergence(self):
        plt.figure()
        plt.plot(range(1, len(self.j_hist) + 1), self.j_hist, marker='o')
        plt.title('Convergece of KMeans Objective j')
        plt.xlabel('epochs')
        plt.ylabel('J (Average Squared Distance)')
        plt.grid(True)
        plt.show()

    def plot_clusters(self, X):
        plt.figure()
        colors = ['red', 'blue', 'green', 'purple', 'orange', 'cyan']
        for i in range(self.k):
            pts = X[self.labels == i]
            plt.scatter(pts[:, 0], pts[:, 1], s=50, c=colors[i % len(colors)], label=f"Cluster {i + 1}")
        plt.scatter(self.centroids[:, 0], self.centroids[:, 1], c='black', s=150, marker='x', label='centroids')
        plt.title('K Means clustering Result')
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.legend()
        plt.show()